# Machine Learning Basics

Developed for Jupyter Hub by Hunter Tiner

## Objectives

- Build binary classification models that predict activity/inactivity of small molecules against human aromatase using supervised learning methods.
- Evaluate the performance of the developed models using performance measures.

## 1. Import bioactivity data from PubChem

In this notebook, we will develop a prediction model for small molecule's activity against human aromatase (https://pubchem.ncbi.nlm.nih.gov/protein/EAW77416), which is encoded by the CYP19A1 gene (https://pubchem.ncbi.nlm.nih.gov/gene/1588). The model will predict the activity of a molecule based on the structure of the molecule (represented with molecular fingerprints).

For model development, we will use the Tox21 bioassay data for human aromatase, archived in PubChem (https://pubchem.ncbi.nlm.nih.gov/bioassay/743139).  The bioactivity data presented on this page can be downloaded by clicking the "Download" button available on this page and then read the data into a data frame.  Alternatively, you can directly load the data into a data frame as shown in the cell below.

In [ ]:
url <- 'https://pubchem.ncbi.nlm.nih.gov/assay/pcget.cgi?query=download&record_type=datatable&actvty=all&response_type=save&aid=743139'
df_raw <- read.csv(url)

In [ ]:
head(df_raw)

__Note:__ Lines 0-2 provide the descriptions for each column (data type, descriptions, units, etc).  These rows need be removed.

In [ ]:
df_raw <- df_raw[3:nrow(df_raw),]
head(df_raw)

The column names in this data frame contain white spaces and special characters.  For simplicity, let's rename the columns (no spaces or special characters except for the "_" character.)

In [ ]:
colnames(df_raw)

In [ ]:
names <- c('pc_result_tag', 'sid', 'cid', 'activity_outcome', 'activity_score', 'activity_url', 'assay_data_comment', 'activity_summary', 'antagonist_activity', 
          'antagonist_potency', 'antagonist_efficacy', 'viability_activity', 'viability_potency', 'viability_efficacy', 'sample_source')

In [ ]:
colnames(df_raw) <- names
colnames(df_raw)

## 2. Check the number of compounds for each activity group

First, we need to understand what our data look like.  Especially, we are interested in the activity class of the tested compounds because we are developing a model that classifies small molecules according to their activities against the target.  This information is available in the "**activity_outcome**" and "**activity_summary**" columns.

In [ ]:
if(!require("dplyr", quietly=TRUE)) {
  install.packages("httr", repos="https://cloud.r-project.org/",
         quiet=TRUE, type="binary")
  library("dplyr", quietly=TRUE)
}

In [ ]:
df_raw %>% group_by(activity_outcome) %>% tally()

Based on the data in the **activity_outcome** column, there are 379 actives, 7562 inactives, and 2545 inconclusives.

In [ ]:
df_raw %>% group_by(activity_outcome, activity_summary) %>% tally()

Now, we can see that, in the **activity_summary** column, the inconclusive compounds are further classified into subclasses, which include:

- **active agonist**
- inconclusive
- inconclusive agonist
- inconclusive antagonist
- inconclusive agonist (cytotoxic)
- inconclusive antagonist (cytotoxic)



As implied in the title of this assay record (https://pubchem.ncbi.nlm.nih.gov/bioassay/743139), this assay aims to identify **aromatase inhibitors**.  Therefore, all **active antagonists** (in the activity summary column) were declared to be **active** compounds (in the activity outcome column).

On the other hand, the assay also identified 612 **active agonists** (in the activity summary column), and they are declared to be **inconclusive** (in the activity outcome column).

With that said, "inactive" compounds in this assay means those which are neither active agonists nor active antagonist.

It is important to understand that the criteria used for determining whether a compound is active or not in a given assay are selected by the data source who submitted that assay data to PubChem.  For the purpose of this assignment (which aims to develop a binary classifier that tells if a molecule is active or inactive against the target), we should clarify what we mean by "active" and "inactive".

- **active** : any compounds that can change (either increase or decrease) the activity of the target.  This is equivalent to either **active antagonists** or **active agonists** in the activity summary column.
- **inactive** : any compounds that do not change the activity of the target.  This is equivalent to **inactive** compounds in the activity summary column.

## 3. Select active/inactive compounds for model building

Now we want to select only the active and inactive compounds from the data frame (that is, active agonists, active antagonists, and inactives based on the "activity summary" column).

In [ ]:
df <- df_raw[ which(df_raw$activity_summary=='active agonist' |
                    df_raw$activity_summary=='active antagonist' |
                    df_raw$activity_summary=='inactive'), ]

nrow(df)

In [ ]:
print(length(unique(df$sid)))
print(length(unique(df$cid)))

Note that the number of CIDs is not the same as the number of SIDs.  There are two important potential reasons for this observation.  

First, not all substances (SIDs) in PubChem have associated compounds (CIDs) because some substances failed during structure standardization.  \[Remember that, in PubChem, substances are depositor-provided structures and compounds are unique structures extracted from substances through structure standardization.]  Because our model will use structural information of molecules to predict their bioactivity, we need to remove substances without associated CIDs (i.e., no standardized structures).

Second, some compounds are associated with more than one substances.  In the context of this assay, it means that a compound may be tested multiple times in different samples (which are designated as different substances).  It is not uncommon that different samples of the same chemical may result in conflicting activities (e.g., active agonist in one sample but inactive in another sample).  In this practice, we remove such compounds with conflicting activities.

## 3-(1) Drop substances without associated CIDs.

First, check if there are subtances without associated CIDs.

In [ ]:
sum(is.na(df$cid))

There are 138 records whose "cid" column is NULL, and we want to remove those records.

In [ ]:
#We are using the complete.cases function which will specify which rows have no missing values. We are specifying the cid column

df <- df[complete.cases(df$cid), ]
nrow(df)

In [ ]:
print(length(unique(df$sid)))
print(length(unique(df$cid)))

In [ ]:
sum(is.na(df$cid))   # Check if the NULL values disappeared in the "cid" column

## 3-(2) Remove CIDs with conflicting activities

Now identify compounds with conflicting activities and remove them.

In [ ]:
cid_conflict <- data.frame()
idx_conflict <- data.frame()

for (mycid in unique(df$cid)){
    
    outcomes <- unique(df[df$cid == mycid, 'activity_summary'])
    
    if (length(outcomes) > 1){    
        idx_tmp <- as.data.frame(df[df$cid == mycid,])
        idx_conflict <- rbind(idx_conflict, idx_tmp)
        cid_conflict <- rbind(cid_conflict, mycid)
    }
}

cat("#" , nrow(cid_conflict) , "CIDs with conflicting activities [associated with" , nrow(idx_conflict) , "rows (SIDs).]")

In [ ]:
head(idx_conflict,10)

In [ ]:
#Using anti_join from the dplyr library!
df <- anti_join(df, idx_conflict)

In [ ]:
df %>% group_by(activity_summary) %>% tally()

In [ ]:
print(length(unique(df$sid)))
print(length(unique(df$cid)))

## 3-(3) Remove redundant data

The above code cells \[in 3-(2)] do not remove compounds tested multiple times if the testing results are consistent [e.g., active agonist in all samples (substances)].  The rows corresponding to these compounds are redundant, so we want remove them except for only one row for each compound.

In [ ]:
df <- df[!duplicated(df$cid),]     # remove duplicate rows except for the first occurring row.

print(length(unique(df$sid)))
print(length(unique(df$cid)))

## 3-(4) Adding "numeric" activity classes

In general, the inputs and outputs to machine learning algorithms need to have numerical forms.   
In this practice, the input (molecular structure) will be represented with binary fingerprints, which already have numerical forms (0 or 1).  However, the output (activity) is currently in a string format (e.g., 'active agonist', 'active antagonist').  Therefore, we want to add an additional, 'activity' column, which contains numeric codes representing the active and inactive compounds:
- 1 for actives (either active agonists or active antagonists)
- 0 for inactives

Note that we are merging the two classes "active agonist" and "active antagonist", because we are going to build a binary classifer that distinguish actives from inactives.

In [ ]:
df$activity <- ifelse(df$activity_summary=="inactive", 0,1)

Check if the new column 'activity' is added to (the end of) the data frame.

In [ ]:
head(df,3)

Double-check the count of active/inactive compounds.

In [ ]:
df %>% group_by(activity_summary) %>% tally()

In [ ]:
df %>% group_by(activity) %>% tally()

## 3-(5) Create a smaller data frame that only contains CIDs and activities.

Let's create a smaller data frame that only contains CIDs and activities.  This data frame will be merged with a data frame containing molecular fingerprint information.

In [ ]:
df_activity <- data.frame(df$cid, df$activity)

In [ ]:
head(df_activity,5)

## 4. Download structure information for each compound from PubChem

Now we want to get structure information of the compounds from PubChem (in isomeric SMILES).

In [ ]:
#cids = df.cid.astype(int).tolist()
cids <- c(df$cid)

In [ ]:
chunk_size <- 200
num_cids <- length(cids)

if (num_cids %% chunk_size == 0){
    num_chunks = num_cids / chunk_size
} else {
    num_chunks = num_cids / chunk_size + 1
    }

num_chunks <-  round(num_chunks, digits = 0)

cat("# CIDs = ", num_cids, " | " , "# CID Chunks = ", num_chunks, "(chunked by ", chunk_size, ")")

In [ ]:
df_smiles <- data.frame()

list_dfs <- c()  # temporary list of data frames

for (i in 1:(num_chunks-1)){
    
    idx1 <- chunk_size * (i - 1) + 1
    idx2 <- chunk_size * i
    
    if ( idx2 > num_cids){
        idx2 <- num_cids
    }
      
    cidstr <- paste(cids[idx1:pmin(idx2,length(cids))],collapse=",")
    
    url <- paste('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/',cidstr,'/property/IsomericSMILES/TXT',sep="")
   
    res <- read.csv(url, sep=",", header = FALSE )
    
    list_dfs <- rbind(list_dfs, res)
    
    Sys.sleep(.2)
                      
    if ( i %% 5 == 0 ){
        print(paste("Processing Chunk", i))
        }
    }


df_smiles <- cbind(list_dfs, df$cid)

head(df_smiles)

In [ ]:
nrow(df_smiles)

In [ ]:
colnames(df_smiles) <- c("smiles","cid")
df_smiles <- df_smiles[c("cid", "smiles")]

head(df_smiles, 5)

## 5. Generate MACCS keys from SMILES.

In [ ]:
if (!require("rcdk", quietly=TRUE)) {
 install.packages("rcdk", repos="https://cloud.r-project.org/",
 quiet=TRUE, type="binary")
 library("rcdk")
}
if (!require("fingerprint", quietly=TRUE)) {
 install.packages("fingerprint", repos="https://cloud.r-project.org/",
 quiet=TRUE, type="binary")
 library("fingerprint")
}

In [ ]:
df_smiles$smiles <- as.character(df_smiles$smiles)

mol <- parse.smiles(df_smiles$smiles)

fp <- lapply(mol, function(x)
  as.character(get.fingerprint(x, type = 'maccs',
                               fp.mode = 'bit', verbose=FALSE)))
             
fp <- as.matrix(unlist(fp))

In [ ]:
head(fp)

We split the maccs string into individual columns.

In [ ]:
maccs <- t(as.data.frame(strsplit(fp, NULL)))

We then generate the column names.

In [ ]:
colnames(maccs) <- paste("maccs", formatC(1:ncol(maccs), width=3, flag="0"), sep = "")

## 6. Merge activity data and fingerprint information

In [ ]:
head(df_activity, 3)

In [ ]:
head(maccs, 3)

In [ ]:
df_data <- df_activity
df_data <- cbind(df_data, maccs)
rownames(df_data) <- NULL

In [ ]:
head(df_data, 3)

In Section 5 of the Python lesson there were two CIDs for which the MACCS keys could not be generated.  They need to be removed from **df_data**.

In [ ]:
df_data <- subset(df_data, CID!=28145 & CID!=28127)

Save df_data in CSV for future use.

In [ ]:
write.csv(df_data,'df_data.csv',row.names=FALSE)

## 7. Preparation for model building

### 7-(1) Loading the data into X and y.

In [ ]:
head(df_data, 3)

In [ ]:
x <- df_data[,3:ncol(df_data)]
y <- df_data["Activity"]

In [ ]:
head(x, 3)

In [ ]:
paste(c(nrow(y), sum(y)))          # Number of actives

### 7-(2) Remove zero-variance features

Some features in X are not helpful in distinguishing actives from inactives, because they are set ON for all compounds or OFF for all compounds.  Such features need to be removed because they would consume more computational resources without improving the model.

In [ ]:
which(apply(x, 2, var) == 0)

In [ ]:
paste(c(nrow(x), ncol(x)))  #- Before removal

In [ ]:
x <- x[ - as.numeric(which(apply(x, 2, var) == 0))]

paste(c(nrow(x), ncol(x)))  #- After removal

In this case, four features had zero variances.  Note that one of them is the first bit (maccs000) of the MACCS keys, which is added as a "dummy" to name each of bits 1~166 as maccs001, maccs002, ... maccs166.  

### 7-(3) Train-Test-Split (a 9:1 ratio)

Now split the data set into a training set (90%) and test set (10%).  The training set will be used to train the model.  The developed model will be tested against the test set.

In [ ]:
if (!require("caret", quietly=TRUE)) {
  install.packages("caret", repos="https://cloud.r-project.org/",
                   quiet=TRUE, type="binary")
  library("caret")
}

In [ ]:
#set.seed(Sys.time())  # using this as a seed will produce different results, this would be more useful in "real-world"
set.seed(3100)
test_inds = createDataPartition(y = 1:nrow(y), p = .1, list = F)

# Split data into test/train using indices
x_test = x[test_inds, ]; y_test = y[test_inds]
x_train = x[-test_inds, ]; y_train = y[-test_inds]

paste(c(nrow(x_train), ncol(x_train), nrow(x_test), ncol(x_test), length(y_train), length(y_test)))
paste(c(sum(y_train), sum(y_test)))

### 7-(4) Balance the training set through downsampling

Check the dimension of the training data set.

In [ ]:
length(y_train)
nrow(x_train)
length(x_train)

Check the number of actives and inactives compound.

In [ ]:
paste(c("# inactives : ", length(y_train) - sum(y_train)))
paste(c("# actives : ", sum(y_train)))

The data set is highly imbalanced \[the inactive to active ratio is ~8].  To address this issue, let's downsample the majority class (inactive compounds) to balance the data set.

In [ ]:
# Indicies of each class' observations
idx_inactives <- which(sapply(y_train, function(y_train) 0 %in% y_train))
idx_actives <- which(sapply(y_train, function(y_train) 1 %in% y_train))
                            
# Number of observations in each class
num_inactives <-  length(idx_inactives)
num_actives   <- length(idx_actives)

# Randomly sample from inactives without replacement
# set.seed(Sys.time())
set.seed(0)
idx_inactives_downsampled <- sample(idx_inactives, num_actives, replace = F)

# Join together downsampled inactives with actives
x_train <- rbind(x_train[idx_inactives_downsampled,], x_train[idx_actives,])
y_train <- append(y_train[idx_inactives_downsampled], y_train[idx_actives])

It is noteworthy that **np.vstack** is used for X_train and **np.hstack** is used for Y_train.  The direction of stacking is different because X_train is a 2-D array and y_train is a 1-D array.

Confirm that the downsampled data set has the correct dimension and active/inactive counts.

In [ ]:
paste(c("# inactives : ", length(y_train) - sum(y_train)))
paste(c("# actives : ", sum(y_train)))

In [ ]:
length(y_train)
nrow(x_train)
length(x_train)

# 8. Build a model using the training set.

Now we are ready to build predictive models using machine learning algorithms.  This notebook will use Naive Bayes and decision tree, because they are relatively fast and simple.

In [ ]:
if (!require("e1071", quietly=TRUE)) {
  install.packages("e1071", repos="https://cloud.r-project.org/",
                   quiet=TRUE, type="binary")
  library("e1071")
}
if (!require("caTools", quietly=TRUE)) {
  install.packages("caTools", repos="https://cloud.r-project.org/",
                   quiet=TRUE, type="binary")
  library("caTools")
}

In [ ]:
#up to this point the R has mimicked our Python lessons.
#to make things easier, we're gonna mix it up a bit
# - Hunter T.

y_train <- as.data.frame(y_train)
y_test <- as.data.frame(y_test)

#rejoin our train and test, the y datasets are our predictors
train <- cbind(y_train, x_train)
test <- cbind(y_test, x_test)

#rename the column back to "activity"
names(train)[1] <- "activity"
names(test)[1] <- "activity"

#since Naive Bayes attempts to classify, we need to make sure that "activity"
#is a factor instead of numeric
train$activity <- as.factor(train$activity)
test$activity <- as.factor(test$activity)

## 8-(1) Naive Bayes

In [ ]:
#train the model!
nbModel <- naiveBayes(activity~., data = train)

In [ ]:
#use the model to predict
nbPredict <- predict(nbModel, test[,-1])

In [ ]:
table(pred=nbPredict,true=test$activity)

In [ ]:
#print confusion matrix
#confusionMatrix() will also ouput accuracy, sensitivity, and specificity!
confusionMatrix(nbPredict, test$activity)

## 8-(2) Decision Tree

In [ ]:
if (!require("rpart", quietly=TRUE)) {
  install.packages("rpart", repos="https://cloud.r-project.org/",
                   quiet=TRUE, type="binary")
  library("rpart")
}
if (!require("rpart.plot", quietly=TRUE)) {
  install.packages("rpart.plot", repos="https://cloud.r-project.org/",
                   quiet=TRUE, type="binary")
  library("rpart.plot")
}

We’re using RPART to build our Decision Tree. The rpart package uses the Recursive Partitioning And Regression Trees algorithm.

In [ ]:
# build the tree with the rpart package
tree <- rpart(activity~.,
              data=train,
              method = "class")

In [ ]:
#use rpart.plot to vkisualize our decidion tree
rpart.plot(tree, nn=TRUE)

In [ ]:
#testing
treePredict <- predict(object=tree,test[-1],type="class")

In [ ]:
table(treePredict, test$activity)

In [ ]:
#view the confusion matrix along with other stats on the model
confusionMatrix(treePredict, test$activity)

## 9. Model building through cross-validation

## !!Python below!!!

In the above section, the models were developed using the default values for many optional hyperparamters, which cannot be learned by the training algorithm.  For example, when building a decision tree model, one should specify how the tree should be deep, how many compounds should be allowed in a single leaf, what is the minimum number of compounds in a single leaf, etc.

The cells below demonstrate how to perform hyperparameter optimization through 10-fold cross-validation.  In this example, five values for each of three hyperparameters used in decision tree are considered (max_depth, min_samples_split, and min_samples_leaf), resulting in a total of 125 combination of the parameter values (=5 x 5 x 5).  For each combination, 10 models are generated (through 10-fold cross validation) and the average performance will be tracked.  The goal is to find the parameter value combination that results in the highest average performance score (e.g., 'roc_auc') from the 10-fold cross validation.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
scores = [ 'roc_auc', 'balanced_accuracy' ]

In [ ]:
ncvs = 10

max_depth_range         = np.linspace( 3, 7, num=5, dtype='int32' )
min_samples_split_range = np.linspace( 3, 7, num=5, dtype='int32' )
min_samples_leaf_range  = np.linspace( 2, 6, num=5, dtype='int32' )

param_grid = dict( max_depth=max_depth_range,
                   min_samples_split=min_samples_split_range,
                   min_samples_leaf=min_samples_leaf_range )

clf = GridSearchCV( DecisionTreeClassifier( random_state=0 ),
                    param_grid=param_grid, cv=ncvs, scoring=scores, refit='roc_auc',
                    return_train_score = True, iid=False)

In [ ]:
clf.fit( X_train, y_train )
print("Best parameter set", clf.best_params_)

If necessary, it is possible to look into the performance data for each parameter value combination (stored in **clf.cv_results_**), as shown in the following cell.

In [ ]:
means_1a = clf.cv_results_['mean_train_roc_auc']
stds_1a  = clf.cv_results_['std_train_roc_auc']

means_1b = clf.cv_results_['mean_test_roc_auc']
stds_1b  = clf.cv_results_['std_test_roc_auc']

means_2a = clf.cv_results_['mean_train_balanced_accuracy']
stds_2a  = clf.cv_results_['std_train_balanced_accuracy']

means_2b = clf.cv_results_['mean_test_balanced_accuracy']
stds_2b  = clf.cv_results_['std_test_balanced_accuracy']

iterobjs = zip( means_1a, stds_1a, means_1b, stds_1b,
                means_2a, stds_2a, means_2b, stds_2b, clf.cv_results_['params'] )

for m1a, s1a, m1b, s1b, m2a, s2a, m2b, s2b, params in iterobjs :

    print( "Grid %r : %0.4f %0.04f %0.4f %0.04f %0.4f %0.04f %0.4f %0.04f"
           % ( params, m1a, s1a, m1b, s1b, m2a, s2a, m2b, s2b))

Uncomment the following cell to look into additional performance data stored in cv_result_.

In [ ]:
#print(clf.cv_results_)

It is important to understand that each model built through 10-fold cross-validation during hyperparameter optimization uses only 90% of the compounds in the training set and the remaining 10% is used for testing that model.  After all parameter value combinations are evaluated, the best parameter values are selected and used to rebuild a model from **all** compounds in the training set.  **GridSearchCV()** takes care of this last step automatically.  Therefore, there is no need to take an extra step to build a model using **cls.fit()** after hyperparameter optimization.

In [ ]:
y_true, y_pred = y_train, clf.predict( X_train )    # Apply the model to predict the training compound's activity.

In [ ]:
CMat = confusion_matrix( y_true, y_pred )    #-- generate confusion matrix
print(CMat)    # [[TN, FP], 
               #  [FN, TP]]

In [ ]:
acc  = accuracy_score( y_true, y_pred )

sens = CMat[ 1 ][ 1 ] / ( CMat[ 1 ][ 0 ] + CMat[ 1 ][ 1 ] )    # TP / (FN + TP)
spec = CMat[ 0 ][ 0 ] / ( CMat[ 0 ][ 0 ] + CMat[ 0 ][ 1 ] )    # TN / (TN + FP )
bacc = (sens + spec) / 2

y_score = clf.predict_proba( X_train )[:, 1]
auc = roc_auc_score( y_true, y_score )

In [ ]:
print("#-- Accuracy          = ", acc)
print("#-- Balanced Accuracy = ", bacc)
print("#-- Sensitivity       = ", sens)
print("#-- Specificity       = ", spec)
print("#-- AUC-ROC           = ", auc)

Compare these performance data with those from section 8-(2) (for the training set).  When the default values were used, the DT model gave >0.99 for all performance measures, but the current models (developed using hyperparameter optimization) have much lower values, ranging from 0.73 to 0.83.  Again, however, what really matters is the performance against the test set, which contains the data not used for model training. 

In [ ]:
y_true, y_pred = y_test, clf.predict(X_test)    #-- Apply the model to predict the test set compounds' activity.

In [ ]:
CMat = confusion_matrix( y_true, y_pred )    #-- generate confusion matrix
print(CMat)    # [[TN, FP], 
               #  [FN, TP]]

In [ ]:
acc  = accuracy_score( y_true, y_pred )

sens = CMat[ 1 ][ 1 ] / ( CMat[ 1 ][ 0 ] + CMat[ 1 ][ 1 ] )
spec = CMat[ 0 ][ 0 ] / ( CMat[ 0 ][ 0 ] + CMat[ 0 ][ 1 ] )
bacc = (sens + spec) / 2

y_score = clf.predict_proba( X_test )[:, 1]
auc = roc_auc_score( y_true, y_score )

print("#-- Accuracy          = ", acc)
print("#-- Balanced Accuracy = ", bacc)
print("#-- Sensitivity       = ", sens)
print("#-- Specificity       = ", spec)
print("#-- AUC-ROC           = ", auc)

Now we can see that the model from hyperparameter optimization gives better performance data against the test set, compared to the model developed using the default parameter values.  Importantly, the model from hyperparameter optimization shows smaller differences in performance measures between the training and test sets, indicatiing that the issue of outffiting has been alleviated substantially.

# Exercises

In this assignment, we will build predictive models using the same aromatase data.

**step 1** Show the following information to make sure that the activity data in the **df_activity** data frame is still available.

- The first five lines of **df_activity**

In [ ]:
# Write your code in this cell.

- The counts of active/inactive compounds in **df_activity**

In [ ]:
# Write your code in this cell.

**Step 2** Show the following information to make sure the structure data is still available.

- The first five lines of **df_smiles**

In [ ]:
# Write your code in this cell.

- the number of rows of **df_smiles**

In [ ]:
# Write your code in this cell.

**Step 3** Generate the (ECFP-equivalent) circular fingerprints from the SMILES strings.
- Use RDKit to generate 1024-bit-long circular fingerprints.
- Set the radius of the circular fingerprint to 2.
- Store the fingerprints in a data_frame called **df_fps** (along with the CIDs).
- Print the dimension of **df_fps**.
- Show the first five lines of **df_fps**.

In [ ]:
# Write your code in this cell

**Step 4** Merge the **df_activity** and **df_fps** data frames into a data frame called **df_data**
- Join the two data frames using the CID column as keys.
- Remove the rows that have any NULL values (i.e., compounds for which the fingerprints couldn't be generated).
- Print the dimension of **df_data**.
- Show the first five lines of **df_data**.

In [ ]:
# Write your code in this cell.

**Step 5** Prepare input and output data for model building
- Load the fingerprint data into 2-D array (X) and the activity data into 1-D array (y).
- Show the dimension of X and y.

In [ ]:
# Write your code in this cell

- Remove zero-variance features from X (if any).

In [ ]:
# Write your code in this cell.

- Split the data set into training and test sets (90% vs 10%) (using random_state=3100).
- Print the dimension of X and y for the training and test sets.

In [ ]:
# Write your code in this cell.

- Balance the training data set through downsampling.
- Show the number of inactive/active compounds in the downsampled training set.

In [ ]:
# Write your code in this cell.

**Step 6** Building a Random Forest model using the balanced training data set.
- First read the followng documents about random forest (https://scikit-learn.org/stable/modules/ensemble.html#forest and https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier). 
- Use 10-fold cross validation to select the best value for the "n_estimators" parameter that maximizes the **balanced accuracy**.  Test 40 values from 5 to 200 with an increment of 5 (e.g., 5, 10, 15, 20, ..., 190, 195, 200).
- For parameters 'max_depth', 'min_samples_leaf', and 'min_samples_split', use the best values found in Section 9.
- For other parameters, use the default values.
- For each parameter value, print the mean balanced accuracies (for both training and test from cross validation). 

In [ ]:
# Write your code in this cell.

**Step 7** Apply the developed RF model to predict the activity of the **training** set compounds.

- Report the confusion matrix.
- Report the accuracy, balanced accurayc, sensitivity, specificity, and auc-roc.

In [ ]:
# Write your code in this cell.

**Step 8** Apply the developed RF model to predict the activity of the **test** set compounds.

- Report the accuracy, balanced accurayc, sensitivity, specificity, and auc-roc.

In [ ]:
# Write your code in this cell.

**Step 9** Read a recent paper published in *Chem. Res. Toxicol.* (https://doi.org/10.1021/acs.chemrestox.7b00037) and answer the following questions (in no more than five sentences for each question).

- What different approaches did the paper take to develop prediction models (compared to those used in this notebook)?
- How different are the models reported in the paper from those constructed in this paper (in terms of the performance measures)? 
- What would you do to develop models with improved performance?

In [ ]:
#Write your answers in this cell.